Use the De Casteljau algorithm with SLERP on 4 given rotations.

Re-parameterize by rotation angle (i.e. twice the arc-length).

In [ ]:
from IPython.display import HTML

In [ ]:
from matplotlib.animation import FuncAnimation
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from unit_quaternion import Quaternion, UnitQuaternion, slerp

In [ ]:
from helper import angles2quat, plot_rotations, prepare_axis, update_plot

In [ ]:
class BezierSegment:
    
    def __init__(self, q0, q1, q2, q3):
        self.q0 = q0
        self.q1 = q1
        self.q2 = q2
        self.q3 = q3
        
    def evaluate(self, t):
        slerp_1_2 = slerp(self.q1, self.q2, t)
        return slerp(
            slerp(slerp(self.q0, self.q1, t), slerp_1_2, t),
            slerp(slerp_1_2, slerp(self.q2, self.q3, t), t),
            t)
    
    def evaluate_constant_velocity(self, angle):
        return self.evaluate(self._angle2time(angle))
        
    def angular_velocity(self, t):
        slerp_1_2 = slerp(self.q1, self.q2, t)
        one = slerp(slerp(self.q0, self.q1, t), slerp_1_2, t)
        two = slerp(slerp_1_2, slerp(self.q2, self.q3, t), t)
        x, y, z = (two * one.inverse()).log()
        # NB: twice the angle, times 3 because degree 3
        return x * 2 * 3, y * 2 * 3, z * 2 * 3
        
    def angle(self, t):
        
        def angular_speed(time):
            return np.linalg.norm(self.angular_velocity(time))
        
        from scipy import integrate
        value, abserr = integrate.quad(angular_speed, 0, t)
        return value

    def _angle2time(self, angle):
        
        def angle_diff(t):
            return self.angle(t) - angle

        from scipy.optimize import bisect
        return bisect(angle_diff, 0, 1)

In [ ]:
b = BezierSegment(
    angles2quat(0, 0, 0),
    angles2quat(10, 0, 0),
    angles2quat(80, 0, 0),
    angles2quat(90, 0, 0),
)

In [ ]:
b.angular_velocity(0)

In [ ]:
b.angular_velocity(0.5)

In [ ]:
b.angular_velocity(1)

In [ ]:
b.angle(0)

In [ ]:
b.angle(0.5)

In [ ]:
b.angle(1)

In [ ]:
def generate_rotations(b):
    max_angle = b.angle(1)
    for t, angle in zip(np.linspace(0, 1, 100), np.linspace(0, max_angle, 100)):
        yield (
            b.evaluate(t),
            b.evaluate_constant_velocity(angle))

In [ ]:
fig, ax = plt.subplots(subplot_kw=dict(projection='dumb3d'))
collections = prepare_axis(2, ax=ax)
plt.close(fig)

In [ ]:
def ani_func(rot):
    return update_plot(collections, rot)

In [ ]:
ani = FuncAnimation(fig, ani_func, init_func=lambda: None, frames=generate_rotations(b), interval=30)
display(HTML(ani.to_jshtml(default_mode='reflect')))

In [ ]:
b = BezierSegment(
    angles2quat(0, 0, 0),
    angles2quat(0, 90, 0),
    angles2quat(90, -45, 0),
    angles2quat(90, 0, 0),
)

In [ ]:
fig, ax = plt.subplots(subplot_kw=dict(projection='dumb3d'))
collections = prepare_axis(2, ax=ax)
plt.close(fig)

In [ ]:
def ani_func(rot):
    return update_plot(collections, rot)

In [ ]:
ani = FuncAnimation(fig, ani_func, init_func=lambda: None, frames=generate_rotations(b), interval=30)
display(HTML(ani.to_jshtml(default_mode='reflect')))

In [ ]:
b = BezierSegment(
    angles2quat(0, 0, 0),
    angles2quat(10, 0, -179),
    angles2quat(20, 90, 179),
    angles2quat(30, 90, 0),
)

In [ ]:
fig, ax = plt.subplots(subplot_kw=dict(projection='dumb3d'))
collections = prepare_axis(2, ax=ax)
plt.close(fig)

In [ ]:
def ani_func(rot):
    return update_plot(collections, rot)

In [ ]:
ani = FuncAnimation(fig, ani_func, init_func=lambda: None, frames=generate_rotations(b), interval=30)
display(HTML(ani.to_jshtml(default_mode='reflect')))